In [22]:
import cv2
import datetime
import os

In [13]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_eye.xml')
smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_smile.xml')

In [19]:
photo_dir = "fotos_capturadas"
os.makedirs(photo_dir, exist_ok=True)

In [23]:
# Función para detectar rostros y sonrisas
def detect(gray, frame):
    original_frame = frame.copy()  # Copia del frame sin dibujos

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)  # Detecta rostros
    for i, (x, y, w, h) in enumerate(faces, start=1):
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = frame[y:y + h, x:x + w]

        # Detectar sonrisas dentro del rostro detectado
        smiles = smile_cascade.detectMultiScale(roi_gray, 1.8, 20)
        for j, (sx, sy, sw, sh) in enumerate(smiles, start=1):
            # Construir la ruta completa para guardar la foto
            filename = f"foto_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
            filepath = os.path.join(photo_dir, filename)  # Ruta completa
            
            # Guardar la imagen SIN rectángulos ni textos
            cv2.imwrite(filepath, original_frame)
            print(f"Foto guardada en: {filepath}")

            # Dibujar rectángulos y etiquetas en la imagen
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Rostro
            cv2.putText(frame, f"Persona {i}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            
            cv2.rectangle(roi_color, (sx, sy), (sx + sw, sy + sh), (0, 255, 0), 2)  # Sonrisa
            cv2.putText(roi_color, f"Sonrisa {j}", (sx, sy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    return frame


In [24]:
video_capture = cv2.VideoCapture(0)

while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convertir a escala de grises
    canvas = detect(gray, frame)

    cv2.imshow('Detección de Sonrisas', canvas)  # Mostrar el video en tiempo real

    # Presionar 'q' para salir
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la cámara y cerrar ventanas
video_capture.release()
cv2.destroyAllWindows()

Foto guardada en: fotos_capturadas\foto_20250401_091708.png
Foto guardada en: fotos_capturadas\foto_20250401_091708.png
